In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from binance.client import Client

# Autenticación en la API de Binance
api_key = 'GFFbe5wk9HnG4eqS1Wlw54WkHrEExQ11yslR3qx6fFTapoXfvHSgcixcRLrB7Cgy'
api_secret = 'lUJlWrU7EhHR3YdcbTFqtdCwYc8qIw5NRUhqYATQ7DtjujYSO9nLZrPXINsvMdLc'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.futures_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "13 year ago UTC")

# Creación de un dataframe con los datos
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Conversión de los precios y volumen a float
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['close'] = data['close'].astype(float)
data['volume'] = data['volume'].astype(float)

# Normalizar los datos
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data[['open', 'high', 'low', 'close', 'volume']]), columns=['open', 'high', 'low', 'close', 'volume'])

# Procesar los datos para la entrada del modelo
data_scaled['t-24'] = data_scaled['close'].shift(24)

# Calcular indicadores técnicos
data_scaled['RSI'] = ta.momentum.RSIIndicator(data_scaled['close']).rsi()
data_scaled['MACD'] = ta.trend.MACD(data_scaled['close']).macd_diff()
data_scaled['EMA_100'] = ta.trend.EMAIndicator(data_scaled['close'], 100).ema_indicator()
data_scaled['EMA_200'] = ta.trend.EMAIndicator(data_scaled['close'], 200).ema_indicator()
data_scaled['ADX'] = ta.trend.ADXIndicator(data_scaled['high'], data_scaled['low'], data_scaled['close']).adx()
data_scaled['ATR'] = ta.volatility.AverageTrueRange(data_scaled['high'], data_scaled['low'], data_scaled['close']).average_true_range()
data_scaled['Momentum'] = ta.momentum.AwesomeOscillatorIndicator(data_scaled['high'], data_scaled['low']).awesome_oscillator()
data_scaled['Volume'] = data_scaled['volume']

# Eliminar las filas con valores NaN
data_scaled.dropna(inplace=True)

X = np.array(data_scaled[['close', 't-24', 'RSI', 'MACD', 'EMA_100', 'EMA_200', 'ADX', 'ATR', 'Momentum', 'Volume']])
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = np.array(data_scaled['close'])

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),  # Aumenté las unidades en la LSTM
    tf.keras.layers.LSTM(50, return_sequences=False),  # Añadí una segunda capa LSTM
    tf.keras.layers.Dense(50, activation='relu'),  # Añadí una capa densa intermedia
    tf.keras.layers.Dense(1)
])

# Compilar el modelo
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val))  # Aumenté las epochs y ajusté el tamaño del batch

# Guardar el modelo entrenado
model.save('ruta/para/guardar/el/modelo/btc_price_prediction_model_indicators.h5')

/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: divide by zero encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: divide by zero encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:787: RuntimeWarning: invalid value encountered in subtract
  directional_index = 100 * np.abs((dip - din) / (dip + din))


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.